In [1]:
pip install -q langchain langchain-community google-generativeai PyMuPDF tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.3 MB/s eta 0:00:00


In [2]:
import os
from getpass import getpass

GEMINI_API_KEY = getpass("🔐 Enter your Google Gemini API Key (kept secret): ")
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

🔐 Enter your Google Gemini API Key (kept secret): ··········


In [3]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

from google.colab import files
uploaded = files.upload()

pdf_file_path = list(uploaded.keys())[0]
raw_text = extract_text_from_pdf(pdf_file_path)
print("Extracted PDF text.")

Saving SJS Transcript Call.pdf to SJS Transcript Call.pdf
Extracted PDF text.


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len,
    separators=["\n\n", "\n", ".", " ", ""]
)

chunks = text_splitter.split_text(raw_text)
print(f"Document split into {len(chunks)} chunks.")

Document split into 61 chunks.


In [5]:
pip install -U langchain-google-vertexai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0


In [7]:
from google.colab import auth
import vertexai
from langchain_google_vertexai import ChatVertexAI
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

auth.authenticate_user() # Use auth.authenticate_user() for authentication
project_id = "synthetic-trail-457104-n4"
region = "us-central1"
vertexai.init(project=project_id, location=region)

llm = init_chat_model("gemini-2.0-flash-001", model_provider="google_vertexai", temperature=0.3, project=project_id)

prompt_template = PromptTemplate(
    input_variables=["chunk"],
    template="""
You are a financial analyst assistant. Read the following chunk from a financial report and extract investor insights.

Respond in **valid JSON only**, without any explanation. Strictly use the format below:

{{
  "growth_prospects": "summary here",
  "business_changes": "summary here",
  "key_triggers": "summary here",
  "material_effects_on_earnings": "summary here"
}}

Chunk:
{chunk}
"""
)

chain = LLMChain(llm=llm, prompt=prompt_template)

results = []
for i, chunk in enumerate(chunks):
    print(f"Processing chunk {i+1}/{len(chunks)}...")
    output = chain.run(chunk=chunk)
    results.append(output)

/tmp/ipython-input-7-1774896781.py:34: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)
/tmp/ipython-input-7-1774896781.py:39: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output = chain.run(chunk=chunk)


Processing chunk 1/61...
Processing chunk 2/61...
Processing chunk 3/61...
Processing chunk 4/61...
Processing chunk 5/61...
Processing chunk 6/61...
Processing chunk 7/61...
Processing chunk 8/61...
Processing chunk 9/61...
Processing chunk 10/61...
Processing chunk 11/61...
Processing chunk 12/61...
Processing chunk 13/61...
Processing chunk 14/61...
Processing chunk 15/61...
Processing chunk 16/61...
Processing chunk 17/61...
Processing chunk 18/61...
Processing chunk 19/61...
Processing chunk 20/61...
Processing chunk 21/61...
Processing chunk 22/61...
Processing chunk 23/61...
Processing chunk 24/61...
Processing chunk 25/61...
Processing chunk 26/61...
Processing chunk 27/61...
Processing chunk 28/61...
Processing chunk 29/61...
Processing chunk 30/61...
Processing chunk 31/61...
Processing chunk 32/61...
Processing chunk 33/61...
Processing chunk 34/61...
Processing chunk 35/61...
Processing chunk 36/61...
Processing chunk 37/61...
Processing chunk 38/61...
Processing chunk 39/6

In [8]:
import json
from collections import defaultdict

results = []
raw_responses = []

for i, chunk in enumerate(chunks):
    print(f"Processing chunk {i+1}/{len(chunks)}...")
    try:
        output = chain.run(chunk=chunk)
        raw_responses.append(output)
        data = json.loads(output)  # Strictly parse as JSON
        results.append(data)
    except Exception as e:
        print(f"Skipping malformed output:\n{output}\n---")

# Merge Results
merged = defaultdict(str)
for data in results:
    for key, value in data.items():
        if value.strip():
            merged[key] += value.strip() + " "

# Display Summary
import pprint
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(dict(merged))

Processing chunk 1/61...
Skipping malformed output:
```json
{
  "growth_prospects": "Information regarding growth prospects may be discussed in the Q1 FY 2023-24 earnings call transcript.",
  "business_changes": "Information regarding business changes may be discussed in the Q1 FY 2023-24 earnings call transcript.",
  "key_triggers": "Information regarding key triggers may be discussed in the Q1 FY 2023-24 earnings call transcript.",
  "material_effects_on_earnings": "Information regarding material effects on earnings may be discussed in the Q1 FY 2023-24 earnings call transcript."
}
```
---
Processing chunk 2/61...
Skipping malformed output:
```json
{
  "growth_prospects": "SJS Enterprises Q1 FY2024 earnings conference call.",
  "business_changes": "None mentioned in the provided text.",
  "key_triggers": "Q&A session following the presentation.",
  "material_effects_on_earnings": "Details to be discussed in the Q1 FY24 earnings call."
}
```
---
Processing chunk 3/61...
Skipping malfo